In [1]:
import nibabel as nib
import numpy as np
import os
from os import path
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.cluster import DBSCAN
import sys
from zipfile import ZipFile

In [2]:
subnum = 'CC0016_core1'
typ = 'bm'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
outdir='/Users/zeynepenkavi/Downloads/ConteQC'

In [3]:
editp = path.join(basedir, 'sub-'+subnum)
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs', 'cp': 'control.dat'}
vol = typ_dict[typ]

In [4]:
bm_edit_img = nib.load(path.join(editp,'mri/%s.mgz')%(vol))
bm_edit_data = bm_edit_img.get_fdata()

In [5]:
bm_unedit_img = nib.load(path.join(uneditp,'sub-%s/mri/%s.mgz')%(subnum, vol))
bm_unedit_data = bm_unedit_img.get_fdata()

In [6]:
diff_data = bm_unedit_data - bm_edit_data

In [7]:
bm_edit_data.shape

(256, 256, 256)

In [12]:
#Indices of nonzero elements
np.asarray(np.asarray(diff_data != 0).nonzero())
#Slice order is Sag, Axe, Cor

array([[ 71,  71,  71, ..., 176, 176, 176],
       [109, 109, 109, ..., 126, 126, 127],
       [154, 155, 156, ...,  55,  56,  56]])

In [13]:
out = pd.DataFrame(np.asarray(np.asarray(diff_data != 0).nonzero()).T).rename(columns={0:"Sag", 1:"Axe", 2:"Cor"})

In [14]:
out['diff_val'] = diff_data[np.where(diff_data != 0)]

In [15]:
out['Action'] = np.where(out.diff_val>0, "delete voxel", "add voxel")

In [16]:
out['Vol'] = vol

In [17]:
out = out.drop(columns="diff_val")

In [18]:
out = out.sort_values(by=['Cor'])

In [19]:
out = out.reset_index(drop=True)

In [20]:
#All edited voxels (no roll up)
out

,Sag,Axe,Cor,Action,Vol
0,106,103,40,delete voxel,brainmask
1,112,102,40,delete voxel,brainmask
2,112,101,40,delete voxel,brainmask
3,111,102,40,delete voxel,brainmask
4,111,101,40,delete voxel,brainmask
...,...,...,...,...,...
700,174,110,161,delete voxel,brainmask
701,168,113,161,delete voxel,brainmask
702,172,113,161,delete voxel,brainmask
703,169,113,161,delete voxel,brainmask


In [21]:
#Split by action before calculating distances
#This should only apply to wm where wm voxels might both be added or removed

#Make distance matrix
dm = distance_matrix(out[['Sag', 'Axe', 'Cor']], out[['Sag', 'Axe', 'Cor']])

In [22]:
#Apply clustering on the distance matrix
#eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other. This is not a maximum bound on the distances of points within a cluster. This is the most important DBSCAN parameter to choose appropriately for your data set and distance function.
#min_samples: The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.
clustering = DBSCAN(eps=50, min_samples=5).fit(dm)

In [23]:
out['cluster'] = clustering.labels_
out

,Sag,Axe,Cor,Action,Vol,cluster
0,106,103,40,delete voxel,brainmask,0
1,112,102,40,delete voxel,brainmask,0
2,112,101,40,delete voxel,brainmask,0
3,111,102,40,delete voxel,brainmask,0
4,111,101,40,delete voxel,brainmask,0
...,...,...,...,...,...,...
700,174,110,161,delete voxel,brainmask,12
701,168,113,161,delete voxel,brainmask,12
702,172,113,161,delete voxel,brainmask,12
703,169,113,161,delete voxel,brainmask,12


In [24]:
out_grouped = out.groupby('cluster')

out_grouped = out_grouped.agg(min_sag=('Sag', min),
               max_sag=('Sag', max),
               min_axe=('Axe', min),
               max_axe=('Axe', max),
               min_cor=('Cor', min),
               max_cor=('Cor', max),
               num_vox=('Cor', 'count')).reset_index()

In [25]:
out_grouped['vol'] = out.Vol.unique()[0]
out_grouped['action'] = out.Action.unique()[0]
out_grouped

,cluster,min_sag,max_sag,min_axe,max_axe,min_cor,max_cor,num_vox,vol,action
0,0,106,112,101,106,40,41,26,brainmask,delete voxel
1,1,138,142,126,131,45,47,22,brainmask,delete voxel
2,2,122,129,120,126,46,47,30,brainmask,delete voxel
3,3,168,176,124,130,52,56,52,brainmask,delete voxel
4,4,132,136,112,118,63,68,30,brainmask,delete voxel
5,5,130,132,100,106,73,82,60,brainmask,delete voxel
6,6,123,128,37,41,83,85,21,brainmask,delete voxel
7,7,142,146,119,123,88,96,55,brainmask,delete voxel
8,8,107,113,122,126,97,100,36,brainmask,delete voxel
9,9,145,151,124,132,98,104,62,brainmask,delete voxel


In [28]:
typ = 'cp'
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs', 'cp': 'control.dat'}
vol = typ_dict[typ]

In [31]:
path.isfile(path.join(editp,'tmp/%s')%(vol))

True

In [27]:
cps = []
with open(fname, 'r') as fd:
    for line in fd:
        line = line.strip()
        vals = line.split(' ')
        if len(vals) == 3:
            cps.append(vals)
cps = np.array(cps, dtype=np.float)

To do still: 
wm
cp
brainsurfs

how will this be used? 
no notes taken while editing
run this for subject after editing to get notes
[present output in slack channel before switching to this fully]

next step i need to do: 
refine previous edits

can this be useful for that?
could at least run all subjects through this and see if i've missed writing down any edits i've made

how should i go about refining edits?
go over mike's reports and compare for each subject how different i am from dorit/mike
run their edited images (taken from box) through 

so timeline:
finish this script
    -go over current output and see if it misses anything for bm
    -write other volume edits extractions
run all subjects through this and see if you've missed recording any edits
move onto refinement of previous edits:
    -download dorit's edits from box and make difference images for 
    -overlay original, my edits difference image, dorit's difference image in freeview and scroll through
move onto editing new subjects